In [10]:
# data_path = 'E:/GitHub/CS-5302-Project-Group-15/Datasets/'

import os 
# PATH = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/'
# os.chdir(PATH)
from IPython.display import Markdown
from python_scripts import llm_rag
from python_scripts import machine_translation
from python_scripts import text_to_speech
from python_scripts import whisper_setup
from python_scripts import llm_rag
from python_scripts import get_audio
from python_scripts import utils

PATH = os.getcwd()
data_path = PATH + '\\Datasets\\MeDAL\\' # maybe CAPS 'data_path' too

In [8]:
# Step 1: Get Query from Audio
audio_path = utils.record_audio()


Press Enter to start recording.
Recording started. Press Enter to stop recording.
Stopping recording...
Recording stopped and saved to C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files\Talha Ahmed_23_M\Talha Ahmed_23_M_Query_3.wav


In [2]:
audio_path = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files\Talha Ahmed_23_M\Talha Ahmed_23_M_Query_3.wav'
os.path.dirname(audio_path)

'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files\\Talha Ahmed_23_M'

In [ ]:
# Step 2: Transcribe Query to English
# List of Whisper models to use for transcription
whisper_models = ["tiny", "base", "small", "medium", "large"]

# Transcribe the audio file using the specified models
transcript = whisper_setup.transcribe_audio(audio_path, whisper_models)

In [ ]:
# Step 3: Feed query into the LLM

In [2]:
# Not an exhaustive list
models = { 
    'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
    'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
    'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
}

In [ ]:
model = llm_rag.DocumentEmbeddingPipeline(
    model_version = models['mixtral'] # change this to experiment with different models available on replicate
)
model.setup_environment()
model.prepare_documents(data_path = data_path, collection_name = "ubaid_notes_2",  joining = True, persistent = False)
model.embed_and_index()

In [ ]:
# response = model.query_data(query = "What is midwifery according to Evans? Explain in detail")
response = []
response = [model.query_data(query = transcript[i]['text']) for i in range(len(whisper_models))]
display(Markdown(f"<b>{response}</b>"))

In [ ]:
# Step 4: Translate it back to the user language

translated_text = machine_translation.translate_text(text = response, src_lang = 'en', trg_lang = transcript[0]['lang'])

In [ ]:
# Step 5: Now speak the response in the user's language
answer_path = utils.generate_answer_path(original_path = audio_path)
text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = answer_path)